# Documentation and Versions of Tape Simulation System
[GitHub:] (https://github.com/sanchita-chakraborty/fermilab_simulating_tapes) 

## Version 1: Simple Queuing System (Completed: 6/9/23)
- Servers are like tellers in a bank
- Tapes work like individuals wanting to use bank services
- Features:
    - Time of arrival of tapes is automated using an inverse sampling formula!
    - Read time for tapes is max speed
    - Mounting speed is fixed
    - Data in server is fixed
    - Each Server can only support one type of tape

## Version 2: Queuing System: Server-Tape Interactions: (Completed 6/12/23)
- Tape characteristics assigned:
    - Tape address: randomized alphanumeric: Done
    - Tape type: randomly chosen for each between LTO8 and LTO9: Done
    - Tape speed: based on LTO8 or LTO9 characteristics: Done
    - Assigned random locations to servers and trays: Done
    - Data stored in tape - randomly assigned between (100,1000): Done
- Read tapes with 10-20% slowdown speed - random number generator of [0.8,0.9]: Done
- Write tapes with 10-20% slowdown speed - random number generator of [0.8,0.9]: Done
- Data is variable - accounted by a randon algorithm: Done
- Search and Sort Algorithm: Done
- Tape Server Relation Changes: DONE
    - Tape:
        - LTO8 and LTO9 characteristics need to be added to the tapes themselves: Done
        - Speed charactersitics need to be moved over to the tape charactersitics: Done
    - Server:
        - LTO8 and LTO9 characteristics need to be removed: DONE
        - Speed characteristics need to be removed: DONE
        - removed server specific service time generation methods since they were redundant: DONE
            - check if it is an IBM server which will need to add shuffle time: Done
- Search and Sort Algorithm Details: Done
    - Give a feeder tape to check for sample run
    - If address mathces, gives location
        - For IBM, does the shuffle algorithm
            - Shuffle algorithm is set to 1 second/frame to shuffle
    - Current Issue: Taking forever to run, even with 10 tapes: RESOLVED

## Version 3: User-Input and Non-Random Choices - (Completed 6/14/23)
- Characteristics Added:
    - Tape has a linear density array - tells you what the linear density of each tape is: Done
        - LTO8 Linear Density = 65620.8645647 bytes/inch
        - LTO9 Linear Density = 68132.2882086 bytes/inch
    - Array which holds the length of each file on a specific tape: Done
    - Pause time - time between files stored on tapes: Done
- Randomized choices changed to user input: Done
- Data pulled from actual sources instead of random numbers: Done
- Clever read/write mimicry algorithm: Done
    - Writing algorithm
        - randomly generates a tape to write to: Done
        - notes down the address and location of this tape: Done
        - randomly generates a file size and then turns it into a length of tape: Done
        - returns write time as well based on write_speed and file size: Done
        - will check if tape is full before writing a file to tape: Done
    - Reading algorithm
        - check the size of the file: Done
        - search for all files of that size as a preliminary check: Done
        - return reading speed of each file and add time appropriately: Done
        - once file found, return time: Done
        - Key Assumption: All files are compressed - affects read/write speed: Done
- Adjustements to arrival and departure algorithms: Done
    - Intention algorithm: is the queue item read/write action
        - first 4 tapes must be write - just for test purposes, as a file can't be read if there are no files stored: Done
- Key Issue: Service times are N/A for everything except for Server 2 - a bit strange. Need to see what is happening here. - FIXED

## Version 4: Realistic Assumptions and Robot Movement - INCOMPLETE
- Files written can be as large as the max data file storable on tape
    - bias it towards smaller file size
- Check if file is compressed or not
- Move the robot to location
- Implement 2 robots per server
- Add LTO6 and LTO7

In [53]:
import numpy as np
import pandas as pd
import random as rand
import string as string

class Tape_Simulation:
    def __init__(self, s1type, s2type, s3type, s4type, s5type):
        self.clock = 0.0 #simulation clock
        
        self.num_in_q = 0 #tape in queue
        self.t_shuffle = 0 #time to shuffle tape in IBM system
        self.t_load = 0 #time to load tape
        self.t_pause = 0.01 #time the tape reader pauses per file read - DEFAULT NUMBER
        self.total_wait_time = 0 #total wait time of each tape
        self.front_frame = 1
        self.write_speed = 1000 # MB/second
        self.tape_types_array = ['LTO8','LTO9']
        self.file_intent_array = ['read','write']
    
        self.num_in_system = 0
        self.number_in_queue = 0
        self.num_arrivals = 0 #number of arrivals
        self.num_departures = 0 #number of departures
        #self.t_arrival = (-np.log(1-(np.random.uniform(low=0.0,high=9))) * 3) #time of next arrival sing Inverse Transform Sampling
        self.t_arrival = rand.uniform(0.01,0.25)
        self.slowdown = rand.uniform(0.8,0.9) # slowdown added to speeds for reading and writing 
        self.t_read_speed = 0 
        
        self.state_S1 = 0 #status of server (binary)
        self.num_of_departures1 = 0
        self.dep_sum_S1 = 0 #total departure time of server
        self.t_departure_S1 = float('inf') #departure time from server
        self.TYPE_S1 = s1type #type of storage server
        
        self.state_S2 = 0 #status of server (binary)
        self.num_of_departures2 = 0
        self.dep_sum_S2 = 0 #total departure time of server
        self.t_departure_S2 = float('inf') #departure time from server
        self.TYPE_S2 = s2type #type of storage server
        
        self.state_S3 = 0 #status of server (binary)
        self.num_of_departures3 = 0
        self.dep_sum_S3 = 0 #total departure time of server
        self.t_departure_S3 = float('inf') #departure time from server
        self.TYPE_S3 = s3type #type of storage server
        
        self.state_S4 = 0 #status of server (binary)
        self.num_of_departures4 = 0
        self.dep_sum_S4 = 0 #total departure time of server
        self.t_departure_S4 = float('inf') #departure time from server
        self.TYPE_S4 = s4type #type of storage server
        
        self.state_S5 = 0 #status of server (binary)
        self.num_of_departures5 = 0
        self.dep_sum_S5 = 0 #total departure time of server
        self.t_departure_S5 = float('inf') #departure time from server
        self.TYPE_S5 = s5type #type of storage server
        
        # robot allotted features
        self.robot_move_speed = 1 #second/tray - Based on youtube video observation
        self.robot_rotate_speed = 0.5 #second/rotation - Based on youtube video observation
        self.t_pull_tape = 1 #second - Based on youtube video observation
        self.t_push_tape = 1 #second - Based on youtube video observation
        self.address_old = [rand.randint(1,5),rand.randint(1,10),rand.randint(1,5)] #initialize robot at a rand location
        if self.address_old[1] == 0 or self.address_old == 1 or self.address_old == 2 or self.address_old == 3 or self.address_old == 4:
            self.col_old = 1
        else:
            self.col_old = 2
        
        # define tape properties
        self.num_tapes = 10
        self.num_files = 1000
        self.LTO8_storage_uncompressed = 1.2e7
        self.LTO8_storage_compressed = 3.0e7
        self.LTO9_storage_uncompressed = 1.8e7
        self.LTO9_storage_compressed = 4.5e7
        self.tape_type = ["" for x in range(self.num_tapes)]
        self.tape_data_stored = [0 for x in range(self.num_tapes)]
        self.assignment = [[0]*3 for m in range(self.num_tapes)]
        self.tape_address_len = 16 #length of address
        self.tape_linear_density = [0 for x in range(self.num_tapes)];
        self.tape_addresses = ["" for x in range(self.num_tapes)] #array of empty addresses for tapes
        self.file_intent = ["" for x in range(self.num_files)] #array of empty addresses for tapes
        self.tape_files = [[-1 for i in range(self.num_files)] for j in range(self.num_tapes)] #array which will hold the length of each file on tape
        self.assign_tape_loc()
        feeder_address = self.tape_addresses[rand.randint(0,len(self.tape_addresses)-1)]
        self.search_sort()
    
    # Algorithms involving interactions with tapes: read, write, open, and close
    def read(self, file_size):
        # Check - see if the size matches any pre-existing file size
        read_time = 0
        for count_tape in range(self.num_tapes):
            details = self.assignment[count_tape]
            reading_speed = details[3]
            for count_file in range(self.num_files):
                read_time += self.tape_files[count_tape][count_file]/reading_speed
                if self.tape_files[count_tape][count_file] == file_size:
                    address_new = self.assignment[count_tape]
                read_time += self.t_pause
                if address_new[0] == self.address_old[0]:
                    if address_new[1] == self.address_old[1]+5 or address_new[1] == self.address_old[1]-5:
                        action = "rotate"
                        move_amt = 0
                        if address_new[1] == 0 or address_new[1] == 1 or address_new[1] == 2 or address_new[1] == 3 or address_new[1] == 4:
                            col_new = 1
                        else:
                            col_new = 2
                    else:
                        if address_new[1] == 0 or address_new[1] == 1 or address_new[1] == 2 or address_new[1] == 3 or address_new[1] == 4:
                            col_new = 1
                        else:
                            col_new = 2
                        if col_new == self.col_old:
                            action = "move"
                            move_amt = abs(address_new[1]-self.address_old[1])
                        else:
                            if col_new == 2:
                                temp_tray = address_new[1]-5
                                move_amt = abs(temp_tray-self.address_old[1])
                            else:
                                temp_tray = address_new[1]-5
                                move_amt = abs(temp_tray-self.address_old[1])
                            action = "move+rotate"
                else:
                    action = "do nothing"
                    move_amt = 0
        robot_time = robot_move(action, move_amt)
        self.address_old = address_new
        self.col_old = col_new
        return read_time+robot_time
    
    def write(self):
        write_time = 0
        indices = [0,0]
        # pick a random tape to write to
        tape_choice = rand.choice(self.assignment)
        
        # find the address of this random tape - [server, server_tray, IBM layer or 0]
        tape_counter = self.assignment.index(tape_choice)
        numrows = len(self.assignment)
        numcols = len(self.assignment[1])
        count = 0
        for i in range(numrows):
            for j in range(numcols):
                if self.assignment[i][j] == -1 and count == 0:
                    indices = [i,j]
                    count += 1
                else:
                    count+=0
        
        file_counter = indices[1]
        address = self.assignment[tape_counter]
        
        # add write time for reading and checking that other file slots in the length are full
        if file_counter != 0:
            for count_dispose2 in range(tape_counter):
                for count_dispose in range(file_counter):
                    details = self.assignment[count_dispose2]
                    read_speed = details[3]
                    write_time += self.tape_files[count_dispose2][count_dispose]/read_speed
        
        # randomly generate data for each file
        write_time = 0
        file_data = rand.uniform(1,20000) #randomly generate size of file for each write event (in MBs) between 1 MBs to 20 GBs
        while True:
            if check(tape_counter) == 0:
                tape_file_length = file_data/self.tape_linear_density[tape_counter]
                break
            else:
                file_data = rand.uniform(1,20000)
        if file_counter != self.num_files:
            self.tape_files[tape_counter][file_counter] = tape_file_length
            if file_counter != self.num_files-1:
                self.tape_files[tape_counter][file_counter+1] = -2
        else:
            print('Too full! Need to initiate different tape sequence')
        write_time += tape_file_length/self.write_speed
        return write_time
    
    def check_tape_storage(self,tape_number):
        storage = 0
        for i in range(self.num_files):
            if self.tape_files[tape_number][i] != -2 and self.tape_files[tape_number][i] != -1:
                storage += self.tape_files[tape_number][i]
            else:
                storage += 0
        if self.tape_type[tape_number] == "LTO8":
            if storage > self.LTO8_storage_uncompressed:
                return -1
            else:
                return 0
        else:
            if storage > self.LTO9_storage_uncompressed:
                return -1
            else:
                return 0
    
    def robot_move(self, action, move_dist):
        #do something to move robot
        if action == "move+rotate":
            move_time = self.robot_move_speed*move_dist
            rotate_time = 1
        elif action == "move":
            move_time = self.robot_move_speed*move_dist
            rotate_time = 0
        elif action == "rotate":
            rotate_time = 1
            move_time = 0
        else:
            rotate_time = 0
            move_time = 0
        return (move_time+rotate_time)
    
    # simple search and sort algorithm for Layer
    def search_sort(self):
        feeder_address = self.tape_addresses[rand.randint(0,len(self.tape_addresses)-1)]
        for k in range(self.num_tapes):
            if feeder_address == self.tape_addresses[k]:
                location = self.assignment[k]
                if location[2] == 3:
                    if self.IBM_layer != 0 and self.IBM_layer != self.front_frame:
                        self.shuffle()
            else:
                location = -1
                self.t_shuffle = 0
        return location
    
    #assign location to all tapes
    def assign_tape_loc(self):
        for j in range(self.num_tapes):
            self.tape_addresses[j] = ''.join(rand.choices(string.ascii_letters + string.digits, k=self.tape_address_len)) #random alphanumeric addresses for tapes of specified length
            self.tape_type[j] = rand.choice(self.tape_types_array)
            for j2 in range(self.num_files):
                if j2 >= 0 and j2 < 3:
                    self.file_intent[j2] = 'write'
                else:
                    self.file_intent[j2] = rand.choice(self.file_intent_array)
            if self.tape_type[j] == 'LTO8':
                self.tape_linear_density[j] = 65620.8645647 #linear density in bytes/inch for LTO8
            else:
                self.tape_linear_density[j] = 68132.2882086 #linear density in bytes/inch for LTO9
            self.server_assignment = rand.randint(1,5) #generates a random server to be assigned to for each tape
            self.server_tray_assignment = rand.randint(1,10)
            self.tape_data_stored[j] = rand.randint(100,1000)
            if self.server_assignment == 3: #IBM Server
                self.IBM_layer = rand.randint(1,5)
            else:
                self.IBM_layer = 0
            if self.tape_type[j] == 'LTO8':
                self.slowdown = rand.uniform(0.8,0.9) # slowdown added to speeds for reading and writing 
                if self.server_assignment == 1:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
                elif self.server_assignment == 2:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
                elif self.server_assignment == 3:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
                elif self.server_assignment == 4:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
                else:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
            else: #LTO9
                self.slowdown = rand.uniform(0.8,0.9) # slowdown added to speeds for reading and writing 
                if self.server_assignment == 1:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
                elif self.server_assignment == 2:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
                elif self.server_assignment == 3:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
                elif self.server_assignment == 4:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
                else:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
            self.assignment[j] = [self.server_assignment, self.server_tray_assignment, self.IBM_layer, self.t_read_speed]
        return self.assignment
    
    #timing routine        
    def time_adv(self,tape_count,file_count):
        t_next_event = min(self.t_departure_S1, self.t_departure_S2, self.t_departure_S3, self.t_departure_S4, self.t_departure_S5, self.t_arrival)
        self.total_wait_time += (self.num_in_q*(t_next_event-self.clock))
        self.clock = t_next_event
        
        if self.t_arrival < self.t_departure_S1 and self.t_arrival < self.t_departure_S2 and self.t_arrival < self.t_departure_S3 and self.t_arrival < self.t_departure_S4 and self.t_arrival < self.t_departure_S5:
            self.arrival(tape_count,file_count)
        elif self.t_departure_S1 < self.t_arrival and self.t_departure_S1 < self.t_departure_S2 and self.t_departure_S1 < self.t_departure_S3 and self.t_departure_S1 < self.t_departure_S4 and self.t_departure_S1 < self.t_departure_S5:
            self.S1(tape_count,file_count)
        elif self.t_departure_S2 < self.t_arrival and self.t_departure_S2 < self.t_departure_S1 and self.t_departure_S2 < self.t_departure_S3 and self.t_departure_S2 < self.t_departure_S4 and self.t_departure_S2 < self.t_departure_S5:
            self.S2(tape_count,file_count)
        elif self.t_departure_S3 < self.t_arrival and self.t_departure_S3 < self.t_departure_S1 and self.t_departure_S3 < self.t_departure_S2 and self.t_departure_S3 < self.t_departure_S4 and self.t_departure_S3 < self.t_departure_S5:
            self.S3(tape_count,file_count)
        elif self.t_departure_S4 < self.t_arrival and self.t_departure_S4 < self.t_departure_S1 and self.t_departure_S4 < self.t_departure_S2 and self.t_departure_S4 < self.t_departure_S3 and self.t_departure_S4 < self.t_departure_S5:
            self.S4(tape_count,file_count)
        else:
            self.S5(tape_count,file_count)
    
    def shuffle(self):
        if self.IBM_layer == self.front_frame+1 or self.IBM_layer == self.front_frame-1:
            self.t_shuffle = 1
        elif self.IBM_layer == self.front_frame+2 or self.IBM_layer == self.front_frame-2:
            self.t_shuffle = 2
        elif self.IBM_layer == self.front_frame+3 or self.IBM_layer == self.front_frame-3:
            self.t_shuffle = 3
        else:
            self.t_shuffle = 4
        self.front_frame = self.IBM_layer
        return self.t_shuffle
    
    #arrival event
    def arrival(self,tape_count,file_count):
        self.num_arrivals += 1
        self.num_in_system += 1
        count = 0
        status_servers = [self.state_S1, self.state_S2, self.state_S3, self.state_S4, self.state_S5]
        if self.num_in_q == 0:
            if all(element == 1 for element in status_servers):
                self.num_in_q+=1
                self.number_in_queue+=1
                self.t_arrival = self.clock+self.gen_int_arr(counter)
            elif all(element == 0 for element in status_servers):
                if rand.randint(1,5) == 1:
                    self.state_S1 = 1
                    self.dep1= self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S1 += self.dep1
                    self.t_departure_S1 = self.clock + self.dep1
                    self.t_arrival = self.clock + self.t_arrival
                elif rand.randint(1,5) == 2:
                    self.state_S2 = 1
                    self.dep2 = self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S2 += self.dep2
                    self.t_departure_S2 = self.clock + self.dep2
                    self.t_arrival = self.clock + self.t_arrival
                elif rand.randint(1,5) == 3:
                    self.state_S3 = 1
                    self.dep3 = self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S3 += self.dep3
                    self.t_departure_S3 = self.clock + self.dep3
                    self.t_arrival = self.clock + self.t_arrival
                elif rand.randint(1,5) == 4:
                    self.state_S4 = 1
                    self.dep4 = self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S4 += self.dep4
                    self.t_departure_S4 = self.clock + self.dep4
                    self.t_arrival = self.clock + self.t_arrival
                else:
                    self.state_S5 = 1
                    self.dep5 = self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S5 += self.dep5
                    self.t_departure_S5 = self.clock + self.dep5
                    self.t_arrival = self.clock + self.t_arrival
            else:
                for status in range(0,4):
                    if status_servers[status] == 1:
                        count+=1
                if count != 0:
                    seed = rand.randint(1,count)
                else:
                    seed = 0
                if seed == 1:
                    self.state_S1 = 1
                    self.dep1 = self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S1 += self.dep1
                    self.t_departure_S1 = self.clock + self.dep1
                    self.t_arrival = self.clock + self.t_arrival
                elif seed == 2:
                    self.state_S2 = 1
                    self.dep2 = self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S2 += self.dep2
                    self.t_departure_S2 = self.clock + self.dep2
                    self.t_arrival = self.clock + self.t_arrival
                elif seed == 3:
                    self.state_S3 = 1
                    self.dep3 = self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S3 += self.dep3
                    self.t_departure_S3 = self.clock + self.dep3
                    self.t_arrival = self.clock + self.t_arrival
                elif seed == 4:
                    self.state_S4 = 1
                    self.dep4 = self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S4 += self.dep4
                    self.t_departure_S4 = self.clock + self.dep4
                    self.t_arrival = self.clock + self.t_arrival
                else:
                    self.state_S5 = 1
                    self.dep5 = self.gen_service_time(tape_count,file_count)
                    self.dep_sum_S5 += self.dep5
                    self.t_departure_S5 = self.clock + self.dep5
                    self.t_arrival = self.clock + self.t_arrival 
        else:
            self.num_in_q += 1
            self.number_in_queue += 1
            self.t_arrival = self.clock + self.t_arrival
    
    def S1(self,tape_count,file_count):
        self.num_of_departures1 += 1
        if self.num_in_q > 0:
            self.dep1 = self.gen_service_time(tape_count,file_count)
            self.dep_sum_S1 += self.dep1
            self.t_departure_S1 = self.clock + self.dep1
            self.num_in_q -= 1
        else:
            self.t_departure_S1 = float('inf')
            self.state_S1 = 0
    
    def S2(self,tape_count,file_count):
        self.num_of_departures2 += 1
        if self.num_in_q > 0:
            self.dep2 = self.gen_service_time(tape_count,file_count)
            self.dep_sum_S2 += self.dep2
            self.t_departure_S2 = self.clock + self.dep2
            self.num_in_q -= 1
        else:
            self.t_departure_S2 = float('inf')
            self.state_S2 = 0
    
    def S3(self,tape_count,file_count):
        self.num_of_departures3 += 1
        if self.num_in_q > 0:
            self.dep3 = self.gen_service_time(tape_count,file_count)
            self.dep_sum_S3 += self.dep3
            self.t_departure_S3 = self.clock + self.dep3
            self.num_in_q -= 1
        else:
            self.t_departure_S3 = float('inf')
            self.state_S3 = 0
    
    def S4(self,tape_count,file_count):
        self.num_of_departures4 += 1
        if self.num_in_q > 0:
            self.dep4 = self.gen_service_time(tape_count,file_count)
            self.dep_sum_S4 += self.dep4
            self.t_departure_S4 = self.clock + self.dep4
            self.num_in_q -= 1
        else:
            self.t_departure_S4 = float('inf')
            self.state_S4 = 0
    
    def S5(self,tape_count,file_count):
        self.num_of_departures5 += 1
        if self.num_in_q > 0:
            self.dep5 = self.gen_service_time(tape_count,file_count)
            self.dep_sum_S5 += self.dep5
            self.t_departure_S5 = self.clock + self.dep5
            self.num_in_q -= 1
        else:
            self.t_departure_S5 = float('inf')
            self.state_S5 = 0
    
    #Generate Arrival and Service Time from Servers
    def type_check(self,tape_counter,file_counter,type_of_server,action):
        read_time = 0
        write_time = 0
        if type_of_server == 'IBM':
            if self.tape_type[tape_counter] == 'LTO8':
                load_to_ready_time = 15
                rewind_time = 59
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter])
                else:
                    write_time += self.write()
            else:
                load_to_ready_time = 17
                rewind_time = 59
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter])
                else:
                    write_time += self.write()
            return (load_to_ready_time+rewind_time+read_time+write_time+self.t_shuffle)
        else:
            if self.tape_type[tape_counter] == 'LTO8':
                load_to_ready_time = 15
                rewind_time = 59
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter])
                else:
                    write_time += self.write()
            else:
                load_to_ready_time = 17
                rewind_time = 59
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter])
                else:
                    write_time += self.write()
            return (load_to_ready_time+rewind_time+read_time+write_time)
                
    def gen_service_time(self,tape_counter,file_counter):
        address = self.assignment[tape_counter]
        server_number = address[0]
        if server_number == 1:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S1,self.file_intent)
        elif server_number == 2:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S2,self.file_intent)
        elif server_number == 3:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S3,self.file_intent)
        elif server_number == 4:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S4,self.file_intent)
        else:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S5,self.file_intent)
        return time


# RUN - Main Part

# Change server types
S1_type = "inv"
S2_type = "inv"
S3_type = "inv"
S4_type = "inv"
S5_type = "inv"

type_input_S1 = input('Enter whether IBM or NonIBM for Server 1')
while S1_type == "inv":
    if type_input_S1.upper() == "NonIBM".upper() or type_input_S1.upper() == "Non IBM".upper():
        S1_type = 'NonIBM'
    elif type_input_S1.upper() == "IBM".upper():
        S1_type = 'IBM'
    else:
        S1_type = "inv"
        print('Invalid response, respond as Non IBM or IBM')
        type_input_S1 = input('Enter whether IBM or NonIBM for Server 1')
        
type_input_S2 = input('Enter whether IBM or NonIBM for Server 2')
while S2_type == "inv":
    if type_input_S2.upper() == "NonIBM".upper() or type_input_S2.upper() == "Non IBM".upper():
        S2_type = 'NonIBM'
    elif type_input_S2.upper() == "IBM".upper():
        S2_type = 'IBM'
    else:
        S2_type = "inv"
        print('Invalid response, respond as Non IBM or IBM')
        type_input_S2 = input('Enter whether IBM or NonIBM for Server 2')
        
type_input_S3 = input('Enter whether IBM or NonIBM for Server 3')
while S3_type == "inv":
    if type_input_S3.upper() == "NonIBM".upper() or type_input_S3.upper() == "Non IBM".upper():
        S3_type = 'NonIBM'
    elif type_input_S3.upper() == "IBM".upper():
        S3_type = 'IBM'
    else:
        S3_type = "inv"
        print('Invalid response, respond as Non IBM or IBM')
        type_input_S3 = input('Enter whether IBM or NonIBM for Server 3')

type_input_S4 = input('Enter whether IBM or NonIBM for Server 4')
while S4_type == "inv":
    if type_input_S4.upper() == "NonIBM".upper() or type_input_S4.upper() == "Non IBM".upper():
        S4_type = 'NonIBM'
    elif type_input_S1.upper() == "IBM".upper():
        S4_type = 'IBM'
    else:
        S4_type = "inv"
        print('Invalid response, respond as Non IBM or IBM')
        type_input_S4 = input('Enter whether IBM or NonIBM for Server 4')
        
type_input_S5 = input('Enter whether IBM or NonIBM for Server 5')
while S5_type == "inv":
    if type_input_S5.upper() == "NonIBM".upper() or type_input_S5.upper() == "Non IBM".upper():
        S5_type = 'NonIBM'
    elif type_input_S5.upper() == "IBM".upper():
        S5_type = 'IBM'
    else:
        S5_type = "inv"
        print('Invalid response, respond as Non IBM or IBM')
        type_input_S5 = input('Enter whether IBM or NonIBM for Server 5')

df = pd.DataFrame(columns = ['Average interarrival time','Average service time Server 1','Average service time Server 2','Average service time Server 3','Average service time Server 4','Average service time Server 5','Utilization Server 1','Utilization Server 2','Utilization Server 3','Utilization Server 4','Utilization Server 5','Total Data Queued in Line Server','Total average wait time Server'])
for i in range(25): #number of replications
    np.random.seed(i)
    server.__init__(S1_type, S2_type, S3_type, S4_type, S5_type)

    for j in range(server.num_tapes):
        for k in range(server.num_files):
            server.time_adv(j,k)

    if server.num_arrivals != 0:
        avg_inter_time = server.clock/server.num_arrivals
    else:
        avg_inter_time = 0 #invalid time indication
    if server.num_of_departures1 != 0:
        service_time_s1 = server.dep_sum_S1/server.num_of_departures1
    else:
        service_time_s1 = 0
    if server.num_of_departures2 != 0:
        service_time_s2 = server.dep_sum_S2/server.num_of_departures2
    else:
        service_time_se = 0
    if server.num_of_departures3 != 0:
        service_time_s3 = server.dep_sum_S3/server.num_of_departures3
    else:
        service_time_s3 = 0
    if server.num_of_departures4 != 0:
        service_time_s4 = server.dep_sum_S4/server.num_of_departures4
    else:
        service_time_s4 = 0
    if server.num_of_departures5 != 0:
        service_time_s5 = server.dep_sum_S5/server.num_of_departures5
    else:
        service_time_s5 = 0

    if server.clock != 0:
        util_time_s1 = server.dep_sum_S1/server.clock
        util_time_s2 = server.dep_sum_S2/server.clock
        util_time_s3 = server.dep_sum_S3/server.clock
        util_time_s4 = server.dep_sum_S4/server.clock
        util_time_s5 = server.dep_sum_S5/server.clock
    else:
        util_time_s1 = -1
        util_time_s2 = -1
        util_time_s3 = -1
        util_time_s4 = -1
        util_time_s5 = -1
        
    a = pd.Series([avg_inter_time,service_time_s1,service_time_s2,service_time_s3,service_time_s4,service_time_s5,util_time_s1,util_time_s2,util_time_s3,util_time_s4,util_time_s5,server.number_in_queue,server.total_wait_time],index = df.columns)
    df = df.append(a,ignore_index = True)
    
df.to_excel('Results_Version_4.xlsx') 

KeyboardInterrupt: Interrupted by user